In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import pymongo
import xarray as xr

In [2]:
client = MongoClient()
db = client.eddies
collection = db.cchdo

In [3]:
def format(LON):
    if LON < 0:
        LON = LON + 360
    else:
        LON = LON
    return LON

In [4]:
rootdir = '/home/cz2397/data/cchdo-hyd/'

In [5]:
station = 0
overlap = 0
result = []

In [6]:
db.cchdo.create_index([('dat', pymongo.ASCENDING), ('loc', pymongo.ASCENDING)])

u'dat_1_loc_1'

In [7]:
for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files):
        if file.endswith('hy1.nc'):
            url = os.path.join(subdir, file)
            DS = xr.open_dataset(url)
            DAT = DS.time.values[0]
            LAT = DS.latitude.values[0]
            LON = format(DS.longitude.values[0])
            dat_var = np.timedelta64(3, 'D')
            dat_min = pd.to_datetime(DAT - dat_var)
            dat_max = pd.to_datetime(DAT + dat_var)
            lat_var = 0.2
            lat_min = LAT - lat_var
            lat_max = LAT + lat_var
            lon_var = 0.4
            lon_min = LON - lon_var
            lon_max = LON + lon_var
            ft = {
                'dat': {'$gt': dat_min, '$lt': dat_max},
                'loc': {'$within': {'$box': [[lon_min, lat_min], [lon_max, lat_max]]}}
            }
            count = db.cchdo.find(ft).count()
            station = station + 1
            if count != 0:
                overlap = overlap + 1
                result.append(url)

100%|██████████| 152/152 [00:09<00:00, 16.84it/s]


In [8]:
station

39791

In [9]:
overlap

621

In [10]:
result

['/home/cz2397/data/cchdo-hyd/257_06MT20060712_nc_hyd/06MT20060712_00310_00089_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/691_ar07e_g_nc_hyd/ar07e_g_00031_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/691_ar07e_g_nc_hyd/ar07e_g_00031_00002_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/237_a17_nc_hyd/a17_00100_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/237_a17_nc_hyd/a17_00101_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/546_ar12_f_nc_hyd/ar12_f_13014_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/288_06GA20000506_nc_hyd/06GA20000506_00063_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/288_06GA20000506_nc_hyd/06GA20000506_00056_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/643_ar19_a_nc_hyd/ar19_a_00063_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/643_ar19_a_nc_hyd/ar19_a_00049_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/643_ar19_a_nc_hyd/ar19_a_00062_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/643_ar19_a_nc_hyd/ar19_a_00062_00002_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/344_i03_nc_hyd/316N145_8_00456